In [ ]:
import pandas as pd
import numpy as np
import jax.numpy as jnp
import numpyro
import matplotlib.pyplot as plt

from PacTimeOrig.data import DataHandling as dh
from PacTimeOrig.data import DataProcessing as dp
from PacTimeOrig.controllers import simulator as sim
from PacTimeOrig.controllers import models as conmod
from PacTimeOrig.controllers import utils as ut

# from PacTimeOrig.controllers import JaxMod
from PacTimeOrig.data import scripts


Load data and create dataframes

In [ ]:
sess=0
cfgparams={'area':'dACC','session':sess+1,'subj':'H','typeswitch':1,'rbfs':[20,30,40,50,60,70,80],'restarts':6}

Xdsgn, kinematics, sessvars, psth = scripts.monkey_run(cfgparams)


Test control simulator

In [ ]:
trial=3
player_pos=np.vstack((Xdsgn[trial].selfXpos,Xdsgn[trial].selfYpos)).transpose()
pry1_pos=np.vstack((Xdsgn[trial].prey1Xpos,Xdsgn[trial].prey1Ypos)).transpose()
pry2_pos=np.vstack((Xdsgn[trial].prey2Xpos,Xdsgn[trial].prey2Ypos)).transpose()

player_vel=np.vstack((Xdsgn[trial].selfXvel,Xdsgn[trial].selfYvel)).transpose()
pry1_vel=np.vstack((Xdsgn[trial].prey1Xvel,Xdsgn[trial].prey1Yvel)).transpose()
pry2_vel=np.vstack((Xdsgn[trial].prey2Xvel,Xdsgn[trial].prey2Yvel)).transpose()

pry1_accel=np.vstack((Xdsgn[trial].prey1Xaccel,Xdsgn[trial].prey1Yaccel)).transpose()
pry2_accel=np.vstack((Xdsgn[trial].prey2Xaccel,Xdsgn[trial].prey2Yaccel)).transpose()

Prep inputs

In [ ]:
A=1.0
B=1.0/60.0
L1=1.6
L2=0.8
outputs=sim.controller_sim_p(6,player_pos,pry1_pos,pry2_pos,L1,L2,A,B,gpscaler=3)
tmp=ut.make_timeline(outputs)


A, B = conmod.define_system_parameters()
num_rbfs = 30

inputs=conmod.prepare_inputs(A, B, outputs['x'], outputs['uout'], pry1_pos, pry2_pos, tmp, num_rbfs, outputs['x'][:,2:], pry1_vel, pry2_vel, ctrltype='p',usingJax=True)


loss_function = conmod.create_loss_function_p(conmod.generate_rbf_basis)
grad_loss = conmod.compute_loss_gradient(loss_function)
(L1_opt, L2_opt), best_params, best_loss = conmod.outer_optimization_p(inputs, conmod.inner_optimization_p, loss_function, grad_loss,maxiter=30,tolerance=1e-3,opttype='global')

weights=best_params[0:-1][0]
widths=best_params[-1]
w1,w2=conmod.generate_sim_switch(inputs, widths, weights)






# Test position velocity error controller (Sim then fit)

In [ ]:

L1=np.array((1.6,0.4))
L2=np.array((2,0.5))
outputs=sim.controller_sim_pv(6,player_pos,player_vel,pry1_pos,pry1_vel,pry2_pos,pry2_vel,L1,L2,None,None,gpscaler=3)

#Make time
tmp=ut.make_timeline(outputs)


#
num_rbfs = 30


A,B=conmod.define_system_parameters(ctrltype='pv')
inputs=conmod.prepare_inputs(A, B, outputs['x'], outputs['uout'], pry1_pos, pry2_pos, tmp, num_rbfs, outputs['x'][:,2:], pry1_vel, pry2_vel, ctrltype='pv',usingJax=True)

loss_function = conmod.create_loss_function_pv(conmod.generate_rbf_basis)
grad_loss = conmod.compute_loss_gradient(loss_function)
(L1_opt, L2_opt), best_params, best_loss = conmod.outer_optimization_pv(inputs, conmod.inner_optimization_pv, loss_function, grad_loss,maxiter=30,opttype='global')


weights = best_params[0:-1][0]
widths = best_params[-1]
w1,w2 = conmod.generate_sim_switch(inputs, widths, weights)




integral error controller 

In [ ]:
L1=np.array((4.1,0.4,1.4))
L2=np.array((2,0.5,0.1))
outputs=sim.controller_sim_pvi(6,player_pos,player_vel,pry1_pos,pry1_vel,pry2_pos,pry2_vel,L1,L2,A=None,B=None,gpscaler=4)



#Make time
tmp=ut.make_timeline(outputs)


#
num_rbfs = 50


A,B=conmod.define_system_parameters(ctrltype='pvi')
inputs=conmod.prepare_inputs(A, B, outputs['x'], outputs['uout'], pry1_pos, pry2_pos, tmp, num_rbfs, outputs['x'][:,2:], pry1_vel, pry2_vel, ctrltype='pvi',usingJax=True)



loss_function = conmod.create_loss_function_pvi(conmod.generate_rbf_basis)
grad_loss = conmod.compute_loss_gradient(loss_function)
(L1_opt_local, L2_opt_local), best_params, best_loss = conmod.outer_optimization_pvi(inputs, conmod.inner_optimization_pvi, loss_function, grad_loss,maxiter=30,opttype='local')


weights = best_params[0:-1][0]
widths = best_params[-1]
w1,w2 = conmod.generate_sim_switch(inputs, widths, weights)





# simulate control with P,V,I,F errors

In [ ]:

L1=np.array((4.1,0.4,1.4,2))
L2=np.array((2,0.5,0.1,1))
outputs=sim.controller_sim_pvif(6, player_pos, player_vel, pry1_pos, pry1_vel,pry1_accel, pry2_pos, pry2_vel, pry2_accel, L1, L2, A=None, B=None,dt=1.0/60.0,Ie_min=-10.0,Ie_max=10.0, gpscaler=3)

#Make time
tmp=ut.make_timeline(outputs)

#
num_rbfs = 24

A,B=conmod.define_system_parameters(ctrltype='pvi')
inputs=conmod.prepare_inputs(A, B, outputs['x'], outputs['uout'], pry1_pos, pry2_pos, tmp, num_rbfs, outputs['x'][:,2:], pry1_vel, pry2_vel, pry1_accel,pry2_accel,ctrltype='pvif',usingJax=True)


loss_function = conmod.create_loss_function_pvif(conmod.generate_rbf_basis)

grad_loss = conmod.compute_loss_gradient(loss_function)

(L1_opt, L2_opt), best_params, best_loss = conmod.outer_optimization_pvif(inputs, conmod.inner_optimization_pvif, loss_function, grad_loss,maxiter=30,opttype='global')

weights = best_params[0:-1][0]
widths = best_params[-1]
w1,w2 = conmod.generate_sim_switch(inputs, widths, weights)